In [ ]:
# This takes photometry of stars through apertures arranged on a grid which moves
# to compensate for poor telescope guiding

# Created 2021 Aug. 20 by E.S.

In [5]:
import pandas as pd
import numpy as np
#from astropy.stats import sigma_clipped_stats
#from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture, aperture_photometry
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import matplotlib.pyplot as plt
from astropy.io import fits, ascii
import glob
import os
from numpy.linalg import inv
import svdt

%matplotlib qt

In [2]:
stem_data = "/Users/bandari/Documents/git.repos/kriz_projects/data/2021-08-04/00_dark_subtracted_flat_fielded/"
stem_star_positions = "/Users/bandari/Documents/git.repos/kriz_projects/notebooks_for_development/output_other/"

In [3]:
# names of files

file_names_stars = sorted(glob.glob(stem_data + "V1432_Aql-*fit"))

file_names_pos = sorted(glob.glob(stem_star_positions + "V1432_Aql-*dat"))

# file of measured displacements between images
file_name_disp = "displacements_v1432_aql.csv"

In [6]:
# machinery to find rotation and translation with SVD:

# Matrix of empirically found positions        F: [ [x_1' y_1' z_1'=0], [x_2' y_2' z_2'=0], [x_3' y_3' z_3'=0] ]
# Rotation matrix                              R
# Matrix of estimated, translated positions    T: [ [x_1 y_1 z_1=0], [x_2 y_2 z_2=0], [x_3 y_3 z_3=0] ]

# fake data
T_estimate = np.array([[0.5, 0.5*np.sqrt(3)/2., 0.], [-0.5*np.sqrt(3)/2., 0.5, 0.], [-0.5, -0.5*np.sqrt(3)/2.,0.]])
F = np.array([[0, 1., 0],[-1, 0, 0],[0, -1, 0]])


R, L, RMSE = svdt.svd(A=T_estimate, B=F)

print(R)
print(L)
print(RMSE)

[[ 0.65465367 -0.75592895  0.        ]
 [ 0.75592895  0.65465367  0.        ]
 [ 0.          0.          1.        ]]
[-1.12854057e-01 -2.77555756e-17  0.00000000e+00]
0.18428990404026965


In [69]:
def coords_brightest(star_positions_scrambled_ids_pass, num_rank):
    '''
    Return the coordinates of the brightest stars in the dataframe (column "Flux")
    
    INPUTS:
    star_positions_scrambled_ids_pass: DataFrame with xcentroid, ycentroid, and flux info
    num_rank: Nth brightest stars returned (i.e., 3 -> coordinates of 3 brightest are returned)
    
    OUTPUTS:
    2D array of x,y coordinates of the Nth brightest stars
    '''
    
    brightest_fluxes = np.sort(star_positions_scrambled_ids_pass["flux"])[-int(num_rank):]
    print("brightest")
    print(brightest_fluxes)
    # pick table rows where stars are at least as bright as the third-brightest star
    idx_cond = star_positions_scrambled_ids_pass["flux"]>=brightest_fluxes[0]
    # get their positions
    x_brights = star_positions_scrambled_ids_pass["xcentroid"][idx_cond]
    y_brights = star_positions_scrambled_ids_pass["ycentroid"][idx_cond]
    #print(star_positions_scrambled_ids_pass[idx_cond])
    coords_empir_brights = zip(x_brights,y_brights)
    
    return list(coords_empir_brights)

In [72]:
# read in image displacements for ALL stars in ALL the frames
disp_images = pd.read_csv(stem_star_positions+file_name_disp)
# read in first image's star locations (but with scrambled IDs)
initial_star_pos = pd.read_csv(file_names_pos[0], delim_whitespace=True)
print(initial_star_pos)

# find the coordinates of the brightest three stars in the initial image;
# we will use this to figure out the net translation and rotation of successive frames
initial_coords_empir_brights = coords_brightest(initial_star_pos,num_rank=3) 
print(initial_coords_empir_brights)

      id   xcentroid   ycentroid  sharpness  roundness1  roundness2  npix  \
0      1  350.244650    1.134922   0.737536    0.115468   -0.097848    25   
1      2    0.184945    4.425653   0.833501    0.257649    0.360274    25   
2      3  359.276330    3.286448   0.656850   -0.002110    0.048119    25   
3      4  481.882260    4.414963   0.612140    0.043175    0.239945    25   
4      5    8.322503    9.328839   0.644281    0.260762    0.034834    25   
..   ...         ...         ...        ...         ...         ...   ...   
297  298   39.622687  510.912870   0.888768   -0.426468   -0.407085    25   
298  299   86.378518  510.748100   0.817096   -0.892780   -0.345335    25   
299  300  209.429580  510.910300   0.935884   -0.009261   -0.749369    25   
300  301  284.571140  510.811560   0.736922   -0.271679   -0.453078    25   
301  302  295.409680  510.801900   0.722965   -0.047643   -0.286862    25   

     sky         peak        flux       mag  
0      0    968.97242    3.73

In [80]:
## ## START FOR-LOOP
for num_star_file in range(0,2200,50): #1200,2200 len(file_names_stars)):

    # make copy of DataFrame of initial stars (we will change the x,y coords to make a prediction)
    predicted_star_pos = initial_star_pos.copy(deep=True)

    # read in current star image
    star_image = fits.open(file_names_stars[num_star_file])[0].data

    # read in empirically-found positions and photometry of ALL stars in this frame (but with scrambled IDs)
    star_info_scrambled_ids = pd.read_csv(file_names_pos[num_star_file], delim_whitespace=True)

    # retrieve (x,y) linear displacement vectors found independently for ALL stars in that frame, as was 
    # found by cross-correlating images
    idx = disp_images["file_name"]==str(os.path.basename(file_names_stars[num_star_file]))
    x_disp = disp_images["x_off_pix"].where(idx).dropna().values[0]
    y_disp = disp_images["y_off_pix"].where(idx).dropna().values[0]

    # shift aperture grid of first (i.e., [0]) array of images by the linear displacement; this is a first-order 
    # correction to fit the current frame (n.b. we use this grid and translate it in order to preserve the ID 
    # information of the stars; otherwise, if we re-find the stars in each frame we don't know which is which)
    #print("print(predicted_star_pos) before")
    #print(predicted_star_pos)
    predicted_star_pos["xcentroid"] = initial_star_pos["xcentroid"]-x_disp
    predicted_star_pos["ycentroid"] = initial_star_pos["ycentroid"]-y_disp
    predicted_coords = list(zip(predicted_star_pos["xcentroid"],predicted_star_pos["ycentroid"]))

    # find the predicted coordinates (in this image) of the brightest three stars (as measured in the initial image)
    print("---")
    predicted_coords_brights = coords_brightest(predicted_star_pos,num_rank=3)
    print("predicted_coords_brights")
    print(predicted_coords_brights)
    # find the empirical coordinates of the brightest *twenty* stars in this image
    # (here we choose twenty, and from those we'll match the correct three, because the brightest
    # three stars in the first image are not necessarily the brightest three in every image)
    empirical_coords_brights = coords_brightest(star_info_scrambled_ids,num_rank=20)
    print(empirical_coords_brights)
    print("---")
    
    ## ## START HERE:
    # match the stars by finding the sets of predicted and empirical coordinates that are within 2 pixels

    ## FYI
    '''
    apertures = CircularAperture(predicted_coords, r=4.)
    norm = ImageNormalize(stretch=SqrtStretch())
    plt.imshow(star_image, cmap='Greys', origin='lower', norm=norm,
               interpolation='nearest')
    apertures.plot(color='blue', lw=1.5, alpha=0.5)
    plt.show()
    '''

    '''
    print("initial star pos")
    print(initial_star_pos)
    vectors_translation = np.transpose((np.subtract(initial_star_pos['xcentroid'],x_disp), 
                              np.subtract(initial_star_pos['ycentroid'],y_disp)))

    #print(vectors_translation.shape)
    print("vectors translation")
    print(vectors_translation)
    '''
    '''

    # get predicted (simple translation) coords of the brightest stars in the first frame
    # get empirical coords of the brightest stars in this frame
    df_vectors_translation = pd.DataFrame(vectors_translation, columns=["x_transl", "y_transl"])
    coords_predic_brights = coords_brightest(df_positions_translation)



    # take the three brightest ones

    # get empirical coords of the brightest stars in this frame
    #print(type(star_positions_scrambled_ids))
    coords_empir_brights = coords_brightest(star_positions_scrambled_ids)
    '''
    # get residuals from mapping of the predicted with the empirical coordinates
    # (then we know which star is which)
    #print(initial_coords_empir_brights)
    #print(coords_empir_brights)
    #print(coords_predic_brights)




    #print(star_positions_scrambled_ids.where[star_positions_scrambled_ids["xcentroid"]==brightest_fluxes[0]])["xcentroid"]

    '''


    ## ## fcn to find the positions of the three brightest stars closest to their estimated positions, 
    ## ## and with a brightness that is consistent (brightest stars have IDs 26, 272, 99 [I believe; check])

    file_names_pos
    '''
    ## ## insert SVD functionality here to find residual translation and rotation

    ## ## make the residual transformation to the aperture locations




    '''
    print(positions)

    apertures = CircularAperture(positions, r=4.)


    phot_table = aperture_photometry(star_image, apertures)
    phot_table['aperture_sum'].info.format = '%.8g'  # for consistent table output


    ## ## START TEST
    # convert to DataFrame
    df_phot = phot_table.to_pandas()

    # remove rows with coordinates that put the stars outside the 512x512 frame, or close to edge
    edge_buffer = 15 # pix
    #phot_table = phot_table[~mask]
    idx_edge_xs = np.logical_or(df_phot["xcenter"] > np.subtract(512,edge_buffer), df_phot["xcenter"] < edge_buffer)
    idx_edge_ys = np.logical_or(df_phot["ycenter"] > np.subtract(512,edge_buffer), df_phot["ycenter"] < edge_buffer)
    df_phot_drop_edges = df_phot.drop(df[np.logical_or(idx_edge_xs,idx_edge_ys)].index, inplace = False)

    # sort: brightest stars first
    df_phot_drop_edges = df_phot_drop_edges.sort_values("aperture_sum", axis=0, ascending=False)

    print(df_phot)
    print(df_phot_drop_edges)

    ## ## END TEST
    '''
    # write text file with aperture photometry
    '''
    text_file_name = "output_other/aper_photom_"+str(os.path.basename(file_names_stars[num_star_file])).split(".")[-2]+".dat"
    ascii.write(phot_table, text_file_name, overwrite=False)
    print("Wrote out "+text_file_name)

    # write FYI plot
    norm = ImageNormalize(stretch=SqrtStretch())
    plt.imshow(star_image, cmap='Greys', origin='lower', norm=norm,
               interpolation='nearest')
    apertures.plot(color='blue', lw=1.5, alpha=0.5)
    '''


    ## ## END FOR-LOOP

---
brightest
[240.2055  283.88976 304.68079]
predicted_coords_brights
[(377.49692999999996, 42.27124), (437.26032999999995, 158.11151999999998), (476.31222, 462.25875999999994)]
brightest
[240.2055  283.88976 304.68079]
[(377.49692999999996, 42.27124), (437.26032999999995, 158.11151999999998), (476.31222, 462.25875999999994)]
---
---
brightest
[240.2055  283.88976 304.68079]
predicted_coords_brights
[(375.44692999999995, 39.79124), (435.21032999999994, 155.63152), (474.26222, 459.7787599999999)]
brightest
[236.41522 289.87707 291.28209]
[(375.48978999999997, 39.766090000000005), (435.21806, 155.72875), (474.33289, 459.56917000000004)]
---
---
brightest
[240.2055  283.88976 304.68079]
predicted_coords_brights
[(373.53693, 40.46124), (433.30033, 156.30151999999998), (472.35222000000005, 460.44875999999994)]
brightest
[215.97782 267.12633 277.63636]
[(373.5357, 40.487381), (433.30825999999996, 156.39369), (472.43293, 460.48785999999996)]
---
---
brightest
[240.2055  283.88976 304.68079]


---
brightest
[240.2055  283.88976 304.68079]
predicted_coords_brights
[(319.68692999999996, 55.611239999999995), (379.45032999999995, 171.45152), (418.50222, 475.5987599999999)]
brightest
[188.86573 196.51178 201.75193]
[(320.13158, 55.537969), (481.15757, 447.32545999999996), (417.9167, 476.00972)]
---
---
brightest
[240.2055  283.88976 304.68079]
predicted_coords_brights
[(318.52693, 56.501239999999996), (378.2903299999999, 172.34151999999997), (417.34222, 476.48875999999996)]
brightest
[178.936   188.2968  203.07887]
[(319.01076, 56.416847), (480.09405999999996, 448.19665), (416.74597, 476.84999000000005)]
---
---
brightest
[240.2055  283.88976 304.68079]
predicted_coords_brights
[(317.58692999999994, 56.76124), (377.35033, 172.60152), (416.40222000000006, 476.74875999999995)]
brightest
[170.97123 176.25971 177.18506]
[(318.0281, 56.513155000000005), (479.06694000000005, 448.44822999999997), (415.79612000000003, 477.13482)]
---
---
brightest
[240.2055  283.88976 304.68079]
predicte

In [10]:
from numpy.linalg import inv
a = np.array([[1., 2.], [3., 4.]])
ainv = inv(a)

In [12]:
print(a)

[[1. 2.]
 [3. 4.]]


In [11]:
np.allclose(np.dot(a, ainv), np.eye(2))
np.allclose(np.dot(ainv, a), np.eye(2))

True

In [42]:
'''
# for plotting
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=4.)
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
apertures.plot(color='blue', lw=1.5, alpha=0.5)
'''

In [44]:
# star images
file_names_stars = sorted(glob.glob(stem_data + "V1432_Aql-*fit"))

# darks
file_names_darks = sorted(glob.glob(stem_data + "d*fit"))

# flats
file_names_flats = sorted(glob.glob(stem_data + "flat*fit"))